## 简答题
1. 如果已经在完全相同的训练集上训练了5个不同的模型，并且它们都达到了95%的准确率，是否还有机会获得更好的效果？如果可以，该怎么做？ 如果不行，为什么？
2. 硬投票分类器和软投票分类器有什么区别？
3. 是否可以通过在多个服务器上并行来加速bagging集成的训练？pasting集成呢？提升集成呢？随机森林或堆叠集成呢？
4. 包外评估的好处是什么？
5. 是什么让极端随机树比一般随机森林更加随机？这部分增加的随机性有什么用？极端随机树比一般随机森林快还是慢？
6. 如果AdaBoost集成对训练数据欠拟合，应该调整哪些超参数？怎么调整？
7. 如果梯度提升集成对训练集过拟合，应该提升还是降低学习率

## 编程题
1. 加载MNIST数据集（**8_sklearn做分类** 里有介绍），将其分为一个训练集、一个验证集和一个测试集（例如使用40000个实例训练，10000个实例验证，最后20000个实例测试）。然后训练多个分类器，比如一个随机森林分类器、一个极端随机树分类器和一个SVM。接下来，尝试使用软投票法或者硬投票法将它们组合成一个集成，这个集成在验证集上的表现要胜过它们各自单独的表现。成功找到集成后，在测试集上测试。与单个的分类器相比，它的性能要好多少？


2. 运行上一题中的单个分类器，用验证集进行预测，然后用预测结果创建一个新的训练集：新训练集中的每个实例都是一个向量，这个向量包含所有分类器对于一个图像的一组预测，目标值是图像的类。恭喜，你成功训练了一个混合器，结合第一层的分类器，它们一起构成了一个堆叠集成。现在在测试集上评估这个集成。对于测试集中的每个图像，使用所有的分类器进行预测，然后将预测结果提供给混合器，得到集成的预测。与前面训练的投票分类器相比，这个集成的结果如何？现在再次尝试使用StackingClassifier。你得到了更好的性能吗？如果是这样，为什么？

In [7]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False, parser='auto')
X, y = mnist["data"], mnist["target"]

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

y = y.astype(np.uint8)

X_temp, X_test, y_temp, y_test = train_test_split( X, y, test_size=20000, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=10000, random_state=42, stratify=y_temp)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [3]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier

rf_clf = RandomForestClassifier(n_estimators=200, max_leaf_nodes=32,random_state=42, n_jobs=-1)
et_clf = ExtraTreesClassifier(n_estimators=200, max_leaf_nodes=32,random_state=42, n_jobs=-1)
svm_clf = LinearSVC(C=1.0, random_state=42, max_iter=2000)
from sklearn.calibration import CalibratedClassifierCV
svm_clf_calibrated = CalibratedClassifierCV(svm_clf, cv=3, method='isotonic')

classifiers = [('rf', rf_clf),('et', et_clf), ('svm', svm_clf_calibrated)]

# for name, clf in classifiers:
#     clf.fit(X_train_scaled, y_train)
#     y_train_pred = clf.predict(X_train_scaled)
#     y_val_pred = clf.predict(X_val_scaled)
#     y_test_pred = clf.predict(X_test_scaled)
#
#     train_accuracy = accuracy_score(y_train, y_train_pred)
#     val_acc = accuracy_score(y_val, y_val_pred)
#     test_accuracy = accuracy_score(y_test, y_test_pred)
#     print(f"{name} 在训练集上的准确率: {train_accuracy:.4f},在验证集上的准确率: {val_acc:.4f},在测试集上的准确率: {test_accuracy:.4f}")


In [9]:
voting_clf = VotingClassifier(estimators=classifiers, voting='soft' ,n_jobs = -1)

voting_clf.fit(X_train_scaled, y_train)

# for name, clf in voting_clf.named_estimators_.items():
#     print(name, "在验证集上的准确率:", clf.score(X_val, y_val), "在训练集上的准确率:", clf.score(X_train, y_train), "在测试集上的准确率:", clf.score(X_test, y_test))
#
# print(f"投票分类器 在训练集上的准确率: {voting_clf.score(X_train, y_train)},在验证集上的准确率: {voting_clf.score(X_val, y_val)},在测试集上的准确率: {voting_clf.score(X_test, y_test)}")


rf 在验证集上的准确率: 0.0975 在训练集上的准确率: 0.0975 在测试集上的准确率: 0.0975
et 在验证集上的准确率: 0.2901 在训练集上的准确率: 0.291075 在测试集上的准确率: 0.2877
svm 在验证集上的准确率: 0.3624 在训练集上的准确率: 0.368925 在测试集上的准确率: 0.3662
投票分类器 在训练集上的准确率: 0.257,在验证集上的准确率: 0.2582,在测试集上的准确率: 0.25465


In [11]:
for name, clf in voting_clf.named_estimators_.items():
    print(name, "在验证集上的准确率:", clf.score(X_val_scaled, y_val), "在训练集上的准确率:", clf.score(X_train_scaled, y_train), "在测试集上的准确率:", clf.score(X_test_scaled, y_test))

print(f"投票分类器 在验证集上的准确率: {voting_clf.score(X_val_scaled, y_val)},在训练集上的准确率: {voting_clf.score(X_train_scaled, y_train)},在测试集上的准确率: {voting_clf.score(X_test_scaled, y_test)}")

rf 在验证集上的准确率: 0.8691 在训练集上的准确率: 0.872975 在测试集上的准确率: 0.86935
et 在验证集上的准确率: 0.8537 在训练集上的准确率: 0.861525 在测试集上的准确率: 0.8577
svm 在验证集上的准确率: 0.9053 在训练集上的准确率: 0.93655 在测试集上的准确率: 0.90865
投票分类器 在验证集上的准确率: 0.9133,在训练集上的准确率: 0.936825,在测试集上的准确率: 0.91395


In [12]:
from sklearn.linear_model import LogisticRegression

rf_clf_trained = voting_clf.named_estimators_['rf']
et_clf_trained = voting_clf.named_estimators_['et']
svm_clf_calibrated_trained = voting_clf.named_estimators_['svm']

rf_val_proba = rf_clf_trained.predict_proba(X_val_scaled)
et_val_proba = et_clf_trained.predict_proba(X_val_scaled)
svm_val_proba = svm_clf_calibrated_trained.predict_proba(X_val_scaled)

X_val_stacked = np.hstack([rf_val_proba, et_val_proba, svm_val_proba])
blender = LogisticRegression(max_iter=1000, random_state=42)
blender.fit(X_val_stacked, y_val)

LogisticRegression(max_iter=1000, random_state=42)

In [14]:

rf_test_proba = rf_clf_trained.predict_proba(X_test_scaled)
et_test_proba = et_clf_trained.predict_proba(X_test_scaled)
svm_test_proba = svm_clf_calibrated_trained.predict_proba(X_test_scaled)

X_test_stacked = np.hstack([rf_test_proba, et_test_proba, svm_test_proba])

blender.score(X_test_stacked, y_test)

0.9184

In [5]:
# 运行 55 min
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

stacking_clf = StackingClassifier(estimators=classifiers,final_estimator=LogisticRegression(max_iter=1000, random_state=42),cv=5, n_jobs=-1)
X_train_full, y_train_full = np.vstack((X_train_scaled, X_val_scaled)), np.concatenate((y_train, y_val))

stacking_clf.fit(X_train_full, y_train_full)

StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(max_leaf_nodes=32,
                                                       n_estimators=200,
                                                       n_jobs=-1,
                                                       random_state=42)),
                               ('et',
                                ExtraTreesClassifier(max_leaf_nodes=32,
                                                     n_estimators=200,
                                                     n_jobs=-1,
                                                     random_state=42)),
                               ('svm',
                                CalibratedClassifierCV(cv=3,
                                                       estimator=LinearSVC(max_iter=2000,
                                                                           random_state=42),
                                                       method='isotonic'))],
                   final_estimator=LogisticRegression(max_iter=1000,
                                                      random_state=42),
                   n_jobs=-1)

In [10]:
stacking_clf.score(X_test_scaled, y_test)

0.92225